### 下载和缓存数据集

In [2]:
import hashlib
import os
import tarfile
import zipfile
import requests

DATA_HUB = dict()
DATA_URL = 'http://d2l-data.s3-accelerate.amazonaws.com/'

`download` 函数用于下载 `DATA_HUB` 中对应数据集名字的数据，若已下载过则验证后直接使用

In [16]:
def download(name, cache_dir=os.path.join('.', 'data')):
    assert name in DATA_HUB, f"{name} 不存在于 {DATA_HUB}" # 传入的数据集名字不存在（没有 url 怎么下载？）
    url, sha1_hash = DATA_HUB[name] # DATA_HUB 存储数据集对应链接和 sha1 加密数据
    os.makedirs(cache_dir, exist_ok=True)
    fname = os.path.join(cache_dir, url.split('/')[-1]) # 数据集名字就在链接中拿
    if os.path.exists(fname): # 如果已经下载过就验证 sha1 是否对应 DATA_HUB 中存储的值
        sha1 = hashlib.sha1()
        with open(fname, 'rb') as f:
            while True:
                data = f.read(1048576) # 读 1MB
                if not data: #如果为空（文件读完就结束）
                    break
                sha1.updata(data) # 计算哈希值
        if sha1.hexdigest() == sha1_hash: # hexdigest() 返回 sha1 加密的摘要，如果匹配则证明文件对了（已经下载的文件可以用）
            return fname
    print(f"从{url}下载{fname}")
    r = requests.get(url, stream=True, verify=True)
    with open(fname, 'wb') as f:
        f.write(r.content)
    return fname

`download_extract` 函数用于下载并解压数据集

In [18]:
def download_extract(name, folder=None): # 有问题，folder 没被用到  //TODO
    fname = download(name) # fname 是下载数据集的完整目录
    base_dir = os.path.dirname(fname) # 拿到前置目录部分（非文件名+后缀）
    data_dir, ext = os.path.splitext(fname) # 如 [path/to/dir/filename, .exe]
    if ext == '.zip':
        fp = zipfile.ZipFile(fname, 'r')
    elif ext in ('.tar', '.gz'):
        fp = tarfile.open(fname, 'r')
    else:
        assert False, '只能解压zip/tar'
    fp.extractall(base_dir) # 解压到同级目录
    return os.path.join(base_dir, folder) if folder else data_dir

`download_all` 函数可以下载字典里所有数据

In [20]:
def download_all():
    for name in DATA_HUB:
        download(name)